# 🚀 Quickstart

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/ASEM000/autoform/blob/main/docs/tutorials/00_quickstart.ipynb)

This notebook is a fast, example-first tour of Autoform.

You will learn how to:

- Write a small LLM workflow in Python
- `trace()` it into an IR (intermediate representation)
- Execute it with `call()` / `acall()`
- Debug it with `checkpoint` + `collect`
- Vectorize it with `batch`
- Get semantic gradients with `pullback`
- Run independent calls concurrently with `sched`


In [1]:
!uv pip install --quiet "git+https://github.com/ASEM000/autoform.git"

## 0) Setup

Autoform uses litellm as a backend. You can use models among the following: https://docs.litellm.ai/docs/providers

In [2]:
import os
import time
import warnings

warnings.filterwarnings("ignore", category=UserWarning)

if "OPENAI_API_KEY" not in os.environ:
    os.environ["OPENAI_API_KEY"] = "your_openai_api_key_here"

MODEL = "openai/gpt-4o"

## 1) Autoform primitives: `format` and `concat`

Autoform programs are normal Python functions, but you build them from primitives.
Primitives are traceable, transformable, and have rules for different interpreters.


In [3]:
import autoform as af


def greeting(name: str) -> str:
    return af.format("Hello, {}!", name)


ir_greet = af.trace(greeting)("...")
print(ir_greet)
af.call(ir_greet)("Autoform")

func(%0:IRVar[str]) -> (%1:IRVar[str]) {
  (%1:IRVar[str]) = format(%0:IRVar[str], template='Hello, {}!', keys=())
}


'Hello, Autoform!'

## 2) Add an LLM call

`lm_call` returns a string. `struct_lm_call` returns a typed `Struct` (recommended).

We'll build a small “rewrite in a style” helper as a realistic starting point.


In [4]:
class RewriteOut(af.Struct):
    rewritten: str


def rewrite_in_style(style: str, text: str) -> RewriteOut:
    template = """Rewrite the text in the following style: {style} TEXT: {text}"""
    prompt = af.format(template, style=style, text=text)
    msgs: list[dict] = [dict(role="user", content=prompt)]
    return af.struct_lm_call(msgs, model=MODEL, struct=RewriteOut)


ir_rewrite = af.trace(rewrite_in_style)("formal", "...")
out = af.call(ir_rewrite)("concise, professional", "hey there, can you help me with my account?")
print(out.rewritten)

Subject: Assistance Required with Account

Hello,

I am seeking assistance with my account. Could you please help? 

Thank you.

[Your Name]


## 3) Debugging and observability: `checkpoint` + `collect`

When workflows get multi-step, you need visibility. Checkpoints are the standard pattern:

- `checkpoint(value, key=..., collection=...)` marks a value.
- `collect(collection=...)` captures all checkpoints inside a context.

This is particularly useful for:

- debugging prompt assembly
- caching expensive intermediate results
- multi-agent pipelines


In [5]:
def rewrite_with_checkpoints(style: str, text: str) -> RewriteOut:
    prompt = af.format("STYLE: {s}\nTEXT: {t}", s=style, t=text)
    prompt = af.checkpoint(prompt, key="prompt", collection="debug")

    out = af.struct_lm_call([{"role": "user", "content": prompt}], model=MODEL, struct=RewriteOut)
    out = af.checkpoint(out, key="raw_output", collection="debug")
    return out


ir_dbg = af.trace(rewrite_with_checkpoints)("formal", "...")

In [6]:
with af.collect(collection="debug") as debug_captured:
    out = af.call(ir_dbg)("friendly", "Please reset my password.")


for key in debug_captured:
    print(f"Checkpoint {key}: {debug_captured[key]}")

Checkpoint prompt: ['STYLE: friendly\nTEXT: Please reset my password.']
Checkpoint raw_output: [RewriteOut(rewritten='Oh, it looks like you\'re having trouble with your password! No worries, I\'ve got your back! 😊 Here\'s how you can reset it:\n\n1. **Visit the login page** of the website or app.\n   \n2. **Click on "Forgot Password?"** or a similar option, typically located just below the login fields.\n\n3. **Enter your registered email address** or username when prompted.\n\n4. **Check your email inbox** for a password reset link sent by the service. (If it\'s not there, don\'t forget to check your spam or junk folder!)\n\n5. **Click on the link or follow the instructions** provided in the email to set up a new password.\n\n6. **Choose a strong and memorable password**, ensuring it’s a combination of letters, numbers, and symbols if possible.\n\nIf you\'re still having any trouble, feel free to reach out for further assistance. We\'re here to help! 💬')]


## 4) Vectorization: `batch`

When you have many inputs, you should batch them. `batch(ir, in_axes=...)` produces a new IR
that accepts batched data.

- `True` means “this input position is batched”
- `False` means “broadcast this input to all items”

This is roughly analogous to `jax.vmap`.


In [7]:
batched_ir = af.batch(ir_rewrite, in_axes=(False, True))

styles = "concise"
texts = [
    "i need help resetting my password",
    "can you refund my last invoice?",
    "the app crashes when i click export",
]

outs = af.call(batched_ir)((styles, texts))
print(outs)

rewritten=['Need assistance with password reset.', 'Refund requested for recent invoice?', 'Export click causes crash.']


## 5) Semantic gradients: `pullback`

`pullback(ir)` creates a new IR that maps:

```
(inputs, output_feedback) -> (outputs, input_feedback)
```

Feedback is a _structured_ object matching the output type. The returned input feedback matches
the input structure (same PyTree).

This enables prompt optimization: use output critiques to improve the prompt.


In [8]:
pb_ir = af.pullback(ir_rewrite)


style = "concise"
text = "Please process my refund."
feedback = RewriteOut.model_construct(rewritten="Too verbose. Make it shorter and more direct.")

(out, (grad_style, grad_text)) = af.call(pb_ir)(((style, text), feedback))
print("Output:\n", out.rewritten)
print("Gradient w.r.t. style:\n", grad_style)
print("Gradient w.r.t. text :\n", grad_text)

Output:
 Refund request, please.
Gradient w.r.t. style:
 Replace "Rewrite the text in the following style: concise" with "Make this text more direct and shorter."
Gradient w.r.t. text :
 Replace "Rewrite the text in the following style: concise" with "Make this text more direct and shorter."


## 6) Concurrency: `sched` + async execution

Many LLM workflows have independent subcalls. Autoform can schedule them using `sched(ir)`:

- With `call()` it still runs sequentially (normal Python).
- With `acall()` it runs independent calls concurrently via `asyncio.gather`.

We’ll build a tiny “summarize + extract actions” workflow and schedule it.


In [9]:
class Actions(af.Struct):
    summary: str
    actions: str


def summarize(text: str) -> str:
    prompt = af.format("Summarize in one sentence:\n{text}", text=text)
    return af.lm_call([{"role": "user", "content": prompt}], model=MODEL)


def extract_actions(text: str) -> str:
    prompt = af.format("Extract action items as bullets:\n{text}", text=text)
    return af.lm_call([{"role": "user", "content": prompt}], model=MODEL)


def summarize_and_actions(text: str) -> Actions:
    s = summarize(text)
    a = extract_actions(text)
    return Actions.model_construct(summary=s, actions=a)


ir_sa = af.trace(summarize_and_actions)("...")
scheduled = af.sched(ir_sa)

In [10]:
TEXT = """We shipped the new billing flow. Some users saw duplicate charges.
We rolled back and are auditing payments. Support should proactively reach out.
"""

t1 = time.time()
out1 = af.call(scheduled)(TEXT)
t2 = time.time()
print(f"Sequential summary ({t2 - t1:.2f} s) :\n", out1.summary)

t1 = time.time()
out2 = await af.acall(scheduled)(TEXT)
t2 = time.time()
print(f"Async summary ({t2 - t1:.2f} s) :\n", out2.summary)

Sequential summary (2.26 s) :
 We rolled back the new billing flow due to duplicate charges experienced by some users and are currently auditing payments, with support being advised to proactively contact affected users.
Async summary (1.49 s) :
 We rolled back the new billing flow due to duplicate charges experienced by some users and are auditing payments while support proactively contacts affected individuals.
